In [2]:
import openfisca_france
from openfisca_france.reforms import plfr2014

tax_benefit_system = openfisca_france.init_tax_benefit_system()

reformed_tax_benefit_system = plfr2014.build_reform(tax_benefit_system)

scenario = reformed_tax_benefit_system.new_scenario()

scenario.init_single_entity(
    period = 2013,
    parent1 = dict(
        age = 40,
        salaire_imposable = 13795,
        ),
    )

simulation = scenario.new_simulation(reference = True)
impo = simulation.calculate('impo', '2013')
print impo

reform_simulation = scenario.new_simulation()
reform_impo = reform_simulation.calculate('impo', '2013')
print reform_impo



[ 0.]
[ 0.]


In [3]:
# -*- coding: utf-8 -*-

from datetime import date

from openfisca_france.tests.base import tax_benefit_system
from openfisca_core.tools import assert_near

from openfisca_france_extension_revenu_de_base.cotisations import build_reform


def test():
    reform = build_reform(tax_benefit_system)
    reform_simulation = reform.new_scenario().init_single_entity(
        period = 2014,
        parent1 = dict(
            birth = date(1980, 1, 1),
            salaire_imposable = 12000,
            ),
        parent2 = dict(
            birth = date(1980, 1, 1),
            salaire_imposable = 6000,
            ),
        enfants = [
            dict(
                birth = date(2014, 1, 1),
                ),
            ],
        ).new_simulation(debug = True)

    absolute_error_margin = 0.01

    assert_near(
        reform_simulation.calculate('salsuperbrut'),
        [17051.3046875, 8525.65234375, 0],
        absolute_error_margin = absolute_error_margin,
        )
    assert_near(
        reform_simulation.calculate('cotisations_contributives'),
        [-5141.63378906, -2570.81689453, 0],
        absolute_error_margin = absolute_error_margin,
        )
    assert_near(
        reform_simulation.calculate('nouv_salaire_de_base'),
        [22192.93945312, 11096.46972656, 0],
        absolute_error_margin = absolute_error_margin,
        )
    assert_near(
        reform_simulation.calculate('salaire_de_base'),
        [14825.93261719, 7412.96630859, 0],
        absolute_error_margin = absolute_error_margin,
        )
    assert_near(
        reform_simulation.calculate('salaire_net'),
        [17199.52734375, 8599.76367188, 0],
        absolute_error_margin = absolute_error_margin,
)

ImportError: No module named openfisca_france_extension_revenu_de_base.cotisations

In [ ]:
from __future__ import division

from openfisca_core import reforms
from openfisca_france.model.base import FloatCol, Individus, Variable


# Build function

def build_reform(tax_benefit_system):
    Reform = reforms.make_reform(
        key = 'revenu_de_base_cotisations',
        name = u"Réforme des cotisations pour un Revenu de base",
        reference = tax_benefit_system,
        )

    class cotisations_contributives(Variable):
        column = FloatCol
        entity_class = Individus
        label = u"Nouvelles cotisations contributives"

        def function(self, simulation, period):
            ags = simulation.calculate('ags', period)
            agff_tranche_a_employeur = simulation.calculate('agff_tranche_a_employeur', period)
            apec_employeur = simulation.calculate('apec_employeur', period)
            arrco_tranche_a_employeur = simulation.calculate('arrco_tranche_a_employeur', period)
            assedic_employeur = simulation.calculate('assedic_employeur', period)
            cotisation_exceptionnelle_temporaire_employeur = simulation.calculate(
                'cotisation_exceptionnelle_temporaire_employeur', period)
            fonds_emploi_hospitalier = simulation.calculate('fonds_emploi_hospitalier', period)
            ircantec_employeur = simulation.calculate('ircantec_employeur', period)
            pension_civile_employeur = simulation.calculate('pension_civile_employeur', period)
            prevoyance_obligatoire_cadre = simulation.calculate('prevoyance_obligatoire_cadre', period)
            rafp_employeur = simulation.calculate('rafp_employeur', period)
            vieillesse_deplafonnee_employeur = simulation.calculate('vieillesse_deplafonnee_employeur', period)
            vieillesse_plafonnee_employeur = simulation.calculate('vieillesse_plafonnee_employeur', period)
            allocations_temporaires_invalidite = simulation.calculate('allocations_temporaires_invalidite', period)
            accident_du_travail = simulation.calculate('accident_du_travail', period)
            agff_tranche_a_employe = simulation.calculate('agff_tranche_a_employe', period)
            agirc_tranche_b_employe = simulation.calculate('agirc_tranche_b_employe', period)
            apec_employe = simulation.calculate('apec_employe', period)
            arrco_tranche_a_employe = simulation.calculate('arrco_tranche_a_employe', period)
            assedic_employe = simulation.calculate('assedic_employe', period)
            cotisation_exceptionnelle_temporaire_employe = simulation.calculate(
                'cotisation_exceptionnelle_temporaire_employe', period)
            ircantec_employe = simulation.calculate('ircantec_employe', period)
            pension_civile_employe = simulation.calculate('pension_civile_employe', period)
            rafp_employe = simulation.calculate('rafp_employe', period)
            vieillesse_deplafonnee_employe = simulation.calculate('vieillesse_deplafonnee_employe', period)
            vieillesse_plafonnee_employe = simulation.calculate('vieillesse_plafonnee_employe', period)

            cotisations_contributives = (
                # cotisations patronales contributives dans le prive
                ags +
                agff_tranche_a_employeur +
                apec_employeur +
                arrco_tranche_a_employeur +
                assedic_employeur +
                cotisation_exceptionnelle_temporaire_employeur +
                prevoyance_obligatoire_cadre +  # TODO contributive ou pas
                vieillesse_deplafonnee_employeur +
                vieillesse_plafonnee_employeur +
                # cotisations patronales contributives dans le public
                fonds_emploi_hospitalier +
                ircantec_employeur +
                pension_civile_employeur +
                rafp_employeur +
                # anciennes cot patronales non-contributives classées ici comme contributives
                allocations_temporaires_invalidite +
                accident_du_travail +
                # anciennes cotisations salariales contributives dans le prive
                agff_tranche_a_employe +
                agirc_tranche_b_employe +
                apec_employe +
                arrco_tranche_a_employe +
                assedic_employe +
                cotisation_exceptionnelle_temporaire_employe +
                vieillesse_deplafonnee_employe +
                vieillesse_plafonnee_employe +
                # anciennes cotisations salariales contributives dans le public
                ircantec_employe +
                pension_civile_employe +
                rafp_employe
                )
            return period, cotisations_contributives

    class nouv_salaire_de_base(Variable):
        reference = tax_benefit_system.column_by_name['salaire_de_base']

        # Le salaire brut se définit dans la réforme comme le salaire super-brut auquel
        # on retranche les cotisations contributives

        def function(self, simulation, period):
            period = period.start.period('month').offset('first-of')
            salsuperbrut = simulation.calculate('salsuperbrut', period)
            cotisations_contributives = simulation.calculate('cotisations_contributives', period)

            nouv_salaire_de_base = (
                salsuperbrut -
                cotisations_contributives
                )
            return period, nouv_salaire_de_base

    class nouv_csg(Variable):
        reference = tax_benefit_system.column_by_name['csg_imposable_salaire']

        # On applique une CSG unique à 22,5% qui finance toutes les prestations non-contributives

        def function(self, simulation, period):
            period = period.start.period('month').offset('first-of')
            nouv_salaire_de_base = simulation.calculate('nouv_salaire_de_base', period)

            nouv_csg = (
                -0.225 * nouv_salaire_de_base
                )
            return period, nouv_csg

    class salaire_net(Variable):
        reference = tax_benefit_system.column_by_name['salaire_net']

        # On retire la nouvelle CSG (pas celle qui finance le RDB) pour trouver le nouveau salaire net

        def function(self, simulation, period):
            period = period.start.period('month').offset('first-of')
            nouv_salaire_de_base = simulation.calculate('nouv_salaire_de_base', period)
            nouv_csg = simulation.calculate('nouv_csg', period)

            salaire_net = (
                nouv_salaire_de_base +
                nouv_csg
                )
            return period, salaire_net

    class salaire_imposable(Variable):
        reference = tax_benefit_system.column_by_name['salaire_imposable']

        # Nous sommes partis du nouveau salaire net et par rapport au salaire imposable actuel,
        # nous avons supprimé : les heures sup, la déductibilité de CSG

        def function(self, simulation, period):
            period = period
            hsup = simulation.calculate('hsup', period)
            salaire_net = simulation.calculate('salaire_net', period)
            primes_fonction_publique = simulation.calculate('primes_fonction_publique', period)
            indemnite_residence = simulation.calculate('indemnite_residence', period)
            supp_familial_traitement = simulation.calculate('supp_familial_traitement', period)
            rev_microsocial_declarant1 = simulation.calculate('rev_microsocial_declarant1', period)

            return period, (
                salaire_net +
                primes_fonction_publique +
                indemnite_residence +
                supp_familial_traitement +
                hsup +
                rev_microsocial_declarant1
                )

return Reform()